In [1]:
import networkx as nx
from sklearn.cluster import SpectralClustering
import json
from networkx.readwrite import json_graph
import plotly.graph_objects as go
import numpy as np

In [2]:
# Читаем файл и обрабатываем строки
records = []
with open('examples/tasks-2.rec', 'r', encoding='utf-8') as file:
    record = {}
    for line in file:
        line = line.strip()  # Убираем лишние пробелы и символы перевода строки
        if line:  # Если строка не пустая
            key, value = line.split(': ', 1)  # Разделяем по ": "
            record[key] = value
        else:  # Пустая строка означает конец записи
            if record:  # Если накоплена запись
                records.append(record)
                record = {}  # Очищаем для следующей записи

    if record:  # Добавляем последнюю запись, если файл не заканчивается пустой строкой
        records.append(record)

In [3]:
len(records)

181731

In [4]:
my_records = []
JobIDs = []
names_list = set()
for record in records:
  if 'Name' in record.keys() and record['Name'][0] != '_' and record['Iteration'] == '0 0':
    my_records.append(record)
    JobIDs.append(record['JobId'])
    names_list.add(record['Name'])

In [5]:
len(my_records)

5203

In [6]:
G = nx.DiGraph()

for record in my_records:
    job_id = record["JobId"]
    depends_on = record.get("DependsOn")

    # Добавляем вершину
    G.add_node(job_id, name=record["Name"])

    # Если есть зависимости, добавляем ориентированные рёбра для каждого из DependsOn
    if depends_on:
        # Если DependsOn несколько, разделяем по пробелам или запятой
        dependencies = depends_on.split()  # Разделяем по пробелам, если зависимость в виде списка
        for dep in dependencies:
          if dep in JobIDs:
            G.add_edge(dep, job_id)  # Добавляем ориентированное ребро от зависимой задачи

In [7]:
len(G)

5203

In [8]:
# Функция для разбиения графа на 4 подграфа
def split_graph_into_parts(graph, num_parts=4):
    # Получаем матрицу смежности
    adjacency_matrix = nx.to_numpy_array(graph)
    
    # Применяем spectral clustering для разбиения на части
    clustering = SpectralClustering(n_clusters=num_parts, affinity='precomputed', random_state=42)
    labels = clustering.fit_predict(adjacency_matrix)
    
    # Создаём подграфы для каждого кластера
    subgraphs = []
    for i in range(num_parts):
        nodes_in_cluster = [node for node, label in zip(graph.nodes, labels) if label == i]
        subgraph = graph.subgraph(nodes_in_cluster).copy()
        subgraphs.append(subgraph)
    
    return subgraphs

In [9]:
# Разбиваем граф на 4 подграфа
subgraphs = split_graph_into_parts(G, num_parts=4)

# Печатаем информацию о каждом подграфе
for i, subgraph in enumerate(subgraphs):
    print(f"Подграф {i+1}: {len(subgraph.nodes)} задач, {len(subgraph.edges)} рёбер")
    
    # Преобразуем граф в структуру данных JSON
    data = json_graph.node_link_data(subgraph)

    # Сериализуем эту структуру в строку JSON
    json_data = json.dumps(data, indent=2)

    # Сохраняем в файл
    with open(f"examples/subgr/graph{i+1}.json", "w") as f:
        f.write(json_data)

/Users/lisovskaya_ekaterina/Library/Python/3.9/lib/python/site-packages/sklearn/manifold/_spectral_embedding.py:310: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  adjacency = check_symmetric(adjacency)


Подграф 1: 1359 задач, 1750 рёбер
Подграф 2: 1351 задач, 1738 рёбер
Подграф 3: 1250 задач, 1613 рёбер
Подграф 4: 1243 задач, 1605 рёбер


In [10]:
import csv

In [11]:
G = subgraphs[3]
with open('edges.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Node1', 'Node2'])  # Заголовок

    # Записываем ребра
    for u, v, data in G.edges(data=True):
        writer.writerow([u, v])

In [12]:
name_to_color = {name: f'rgba({np.random.randint(255)}, {np.random.randint(255)}, {np.random.randint(255)}, 0.8)' for name in names_list}


In [13]:
G = subgraphs[3]
print(len(G))

pos = nx.spectral_layout(G)

# Координаты ребер
data = []
annotations = [] 
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    data.append(go.Scatter(
        x=[x0,x1], y=[y0,y1],
        mode='lines',
        line=dict(width=0.5, color='#888'),
        showlegend=True,
        hoverinfo='text',
        text=G.nodes[edge[0]]['name'] + '__&&__' + G.nodes[edge[1]]['name']
        )
    )

    # Вычисляем угол для стрелки (вектора направления)
    arrow_dx = x1 - x0
    arrow_dy = y1 - y0
    arrow_length = ((x1-x0)**2+(y1-y0)**2)**0.5  # Длина стрелки

    # Нормализуем вектор, чтобы получить направление стрелки
    #norm = (arrow_dx**2 + arrow_dy**2)**0.5
    arrow_dx /= arrow_length
    arrow_dy /= arrow_length

    # Добавляем стрелку как аннотацию
    annotations.append(dict(
        x=x1 - arrow_length * arrow_dx,
        y=y1 - arrow_length * arrow_dy,
        ax=x1,
        ay=y1,
        xref="x", yref="y",
        axref="x", ayref="y",
        showarrow=True,
        arrowhead=1,  # Тип стрелки (параметры: 2 — большая, 3 — маленькая)
        arrowsize=1,  # Размер стрелки
        arrowcolor='black'  # Цвет стрелки
    ))

# Координаты узлов
node_x = []
node_y = []
node_name = []
node_colors = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_name.append(G.nodes[node]['name'])
    node_colors.append(name_to_color[G.nodes[node]['name']])

data.append(go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    marker=dict(
        size=10,
        color=node_colors,
        line=dict(width=1, color='black')
    ),
    text=node_name,
    hoverinfo='text',
    showlegend=True
)
)

# Создание фигуры и отображение
fig = go.Figure(data=data,#[edge_trace, node_trace],
                layout=go.Layout(
                    showlegend=False,
                    hovermode='closest',
                    xaxis=dict(showline=False, showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showline=False, showgrid=False, zeroline=False, showticklabels=False),
                    annotations=annotations
                ))

fig.show()

1243


In [14]:
import networkx as nx
import plotly.graph_objects as go

# Граф G и его координаты
G = subgraphs[3]
pos = nx.spectral_layout(G)

# Координаты ребер
data = []
annotations = [] 

# Массив для отслеживания зависимостей
edges_dict = {edge: (G.nodes[edge[0]]['name'], G.nodes[edge[1]]['name']) for edge in G.edges()}

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    data.append(go.Scatter(
        x=[x0, x1], y=[y0, y1],
        mode='lines',
        line=dict(width=0.5, color='#888'),
        showlegend=True,
        hoverinfo='text',
        text=G.nodes[edge[0]]['name'] + '__&&__' + G.nodes[edge[1]]['name'],
        name='edge_' + str(edge),
        line_shape='linear',
    ))

    # Вычисление стрелки
    arrow_dx = x1 - x0
    arrow_dy = y1 - y0
    arrow_length = ((x1 - x0) ** 2 + (y1 - y0) ** 2) ** 0.5

    arrow_dx /= arrow_length
    arrow_dy /= arrow_length

    annotations.append(dict(
        x=x1 - arrow_length * arrow_dx,
        y=y1 - arrow_length * arrow_dy,
        ax=x1,
        ay=y1,
        xref="x", yref="y",
        axref="x", ayref="y",
        showarrow=True,
        arrowhead=1,
        arrowsize=1,
        arrowcolor='black'
    ))

# Координаты узлов
node_x = []
node_y = []
node_name = []
node_colors = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_name.append(G.nodes[node]['name'])
    node_colors.append(name_to_color[G.nodes[node]['name']])

data.append(go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    marker=dict(
        size=10,
        color=node_colors,
        line=dict(width=1, color='black')
    ),
    text=node_name,
    hoverinfo='text',
    name='nodes',
    showlegend=True,
    customdata=node_name  # Добавление customdata для узлов
))

# Обработчик события hover для подсветки зависимых вершин
def hover_callback(trace, points, selector):
    for point in points.point_inds:
        node = trace.customdata[point]
        # Найдем вершины, от которых зависит эта вершина
        dependent_nodes = [n for n in G.neighbors(node) if n != node]
        # Модифицируем цвета узлов
        node_colors_new = [name_to_color[n] if n not in dependent_nodes else 'red' for n in node_name]
        trace.marker.color = node_colors_new
    return trace

# Создание фигуры и добавление обработчиков hover
fig = go.Figure(data=data, layout=go.Layout(
    showlegend=False,
    hovermode='closest',
    xaxis=dict(showline=False, showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showline=False, showgrid=False, zeroline=False, showticklabels=False),
    annotations=annotations
))

# Привязка функции для подсветки узлов при наведении на вершину
fig.data[0].on_hover(hover_callback)  # Установите обработчик на первую scatter-графику

fig.show()
